In [ ]:
%load_ext dotenv
%dotenv

import os 

#user_name = os.environ.get('username')
#password = os.environ.get('password')
#print(user_name, password)

from geo.Geoserver import Geoserver
geo = Geoserver('https://geosrv.earthbyte.org/geoserver/', 
                username=os.environ.get('username'), password=os.environ.get('password'))



In [ ]:
geo.create_workspace(workspace='paleo_topo')

In [ ]:

# list stores
! curl -v -u $username:$password \
    https://geosrv.earthbyte.org/geoserver/rest/workspaces/paleo_topo/coveragestores



In [ ]:
# create coverage store
! curl -v -u $username:$password  -XPOST \
    -H "Content-type: application/json" \
    -d '{"coverageStore":{"name":"test_netcdf_2","type":"NetCDF","enabled":true,"_default":false,"workspace":{"name":"paleo_topo"},"url":"file:placeholder.nc"}}' \
    https://geosrv.earthbyte.org/geoserver/rest/workspaces/paleo_topo/coveragestores



In [ ]:
# update store coverage
! curl -v -u $username:$password -XPUT \
    -H "Content-Type: application/zip" \
    -H "Accept: application/json" \
    --data-binary @paleotopo_1.00d_40.00Ma.nc.zip \
    "https://geosrv.earthbyte.org/geoserver/rest/workspaces/paleo_topo/coveragestores/test_netcdf_2/file.netcdf?coverageName=test_2"
        

In [ ]:
import subprocess
ret=subprocess.run(["ls", "-l", "/dev/null"], capture_output=True)
print(ret.returncode)

In [ ]:
! for i in $(seq 1 10); do echo $i; done

#### build a shell script to do the uploading

In [ ]:
#easier to create a .sh file than calling subprocess from python
import json

with open("upload_netcdf.sh", 'w+') as f:
    for i in range(201,396):
        # create coverage store
        cfg = {
            "coverageStore":{
                "name":f"paleotopo_{i}_Ma",
                "type":"NetCDF",
                "enabled":True,
                "_default":False,
                "workspace":{"name":"paleo_topo"},
                "url":"file:placeholder.nc"
            }
        }
        create_cmd_str = (
            'curl -u $username:$password  -XPOST '
            '-H "Content-type: application/json" '
            '-d '
            f"'{json.dumps(cfg)}' "
            'https://geosrv.earthbyte.org/geoserver/rest/workspaces/paleo_topo/coveragestores'
        )
        #print(create_cmd_str)


        # update store coverage
        update_cmd_str = (
            'curl -u $username:$password -XPUT '
            '-H "Content-Type: application/zip" '
            '-H "Accept: application/json" '
            f'--data-binary @./zipped_for_geoserver/paleotopo_1.00d_{i}.00Ma.nc.zip '
            f'"https://geosrv.earthbyte.org/geoserver/rest/workspaces/paleo_topo/coveragestores/paleotopo_{i}_Ma/file.netcdf?coverageName=paleotopo_{i}_Ma"'
        )

        f.write(create_cmd_str+'\n')
        f.write(update_cmd_str+'\n')
        f.write('sleep 2\n')


print('Done!')


#### run the shell script

In [ ]:
! chmod +x upload_netcdf.sh
! ./upload_netcdf.sh